见5模型架构图， 黄色和红色是k-v，那么group_num就是k-v的数量，group_num=1那么就是，1组维度为hidden_dim的k-v，共享多头维度为hidden_dim的q

核心就是,
（1）需要共享的k-v为：group_num个，每个都是hidden_dim的维度。需要在linear中，将k-v的维度变为group_num*head_dim，
（2）用于共享，需要复制k-v次数为num_head//group_num:expend(batch_size, group_num, self.num_head//group_num, seq_len, self.head_dim).reshape(batch_size, self.num_head//group_num*group_num, seq_len, self.head_dim)

其他与 MHA完全相同

In [1]:
import torch
from torch import nn
class GroupQueryAttention(torch.nn.Module):
    def __init__(self, hidden_size, num_heads, group_num):
        super(GroupQueryAttention, self).__init__()
        self.num_heads = num_heads
        self.head_dim = hidden_size // num_heads
        self.group_num = group_num

        ## 初始化Q、K、V投影矩阵
        self.q_linear = nn.Linear(hidden_size, hidden_size)
        self.k_linear = nn.Linear(hidden_size, self.group_num * self.head_dim) # 每个组有group_num个q。每个q有head_size dim
        self.v_linear = nn.Linear(hidden_size, self.group_num * self.head_dim) # 每个k-v 处理 group_num个

        ## 输出线性层
        self.o_linear = nn.Linear(hidden_size, hidden_size)

    def forward(self, hidden_state, attention_mask=None):
       #hidden_state就是输入的 训练doc句子，shape为 [batch_size, seq_len, hidden_size]
        batch_size = hidden_state.size()[0]

        query = self.q_linear(hidden_state)
        key = self.k_linear(hidden_state)
        value = self.v_linear(hidden_state)

        query = self.split_head(query)
        key = self.split_head(key, self.group_num)
        value = self.split_head(value, self.group_num)

        # 注意力分数的最后两维度 就是 seq_len * seq_len，因为，q-k最后两维为 seq_len * head_dim，而softmax是对每个seq位置进行
        attention_scores = torch.matmul(query, key.transpose(-1, -2)) / torch.sqrt(torch.tensor(self.head_dim))

        if attention_mask != None:
            attention_scores += attention_mask * -1e-9 # 利用 softmax函数的饱和性，实现mask机制

        # atten_probs的形状为 batch_size, num_heads, seq_len, seq_len。最后一维代表的 就是每个位置 对于其他位置的 注意力权重。乘以v，就是最终的结果
        # softmax对 seq_len * seq_len 矩阵的每个位置都softmax激活。包括mask住的位置，softmax后为0。softmax后矩阵仍然为 seq_len * seq_len，再去与 V相乘变为，seq_len * head_dim
        attention_probs = torch.softmax(attention_scores, dim=-1)

       #output的形状为 batch_size, num_heads, seq_len, head_dim
        output = torch.matmul(attention_probs, value)

       # 将多头 合并
        output = output.transpose(-1, -2).contiguous().view(batch_size, -1, self.head_dim * self.num_heads)

       # 增加线性投影
        output = self.o_linear(output)

        return output


    # num_heads 个头 每个头 head_dim维度， 共享group_num个k-v每个k-v 的维度为head_dim，比如：8个头每个头768//8维度，组为2组每组为768//8维度，需要复制4次k-v，被8个头共享
    # 假设x的每个token的维度为 hidden_dim，那么，head_dim = hidden_dim // num_heads，
    def split_head(self, x, group_num=None):

        batch_size,seq_len = x.size()[:2]

        if group_num == None:
            # x 的形状为 batch_size, seq_len, hidden_dim。其中hidden_dim = num_heads * head_dim
            # 返回 batch_size,num_heads, seq_len, head_dim, 其中将输入的hidden_dim均分多头，
            # 均分后的为head_dim 计算注意力
            return x.view(batch_size, -1, self.num_heads, self.head_dim).transpose(1,2)
        else:
            # x为self.k-self.v处理后的 形状为 batch_size, seq_len, group_num*head_dim
            # 将输入x重新 reshape 到指定形状并进行转置操作，以便符合后续处理所需的维度
            x = x.view(batch_size, -1, group_num, self.head_dim).transpose(1,2)

            # 通过维度扩展和重塑，调整x的形状，以适应多头注意力机制的需求
            # `.expand()` 不创建新的数据副本，通过广播机制重复引用原始数据。通过广播机制共享内存中的k-v数据。
            # 共享的 k-v相当于 参数了减少了 self.num_heads // group_num倍，实际计算的时候仍然使用复制的参数构造出self.num_heads个头
            x = x[:, :, None, :, :].expand(batch_size, group_num, # 如果：group_num = 1 为 MQA，意味着q为多个头（假设为8），共享1组相同的k-v（8/1=8复制了8次）。 如果：group_num为num_head为 MHA，意味着q为多头，且k-v也为多头没有发生复制
                                           self.num_heads // group_num, # 假如num_heads为8，group_num为2将k和v都分为2组，每组需要4个k-v来进行 GQA。
                                           seq_len, self.head_dim).reshape # 由于expend复制了内存，导致，内存不连续，所以不能使用view改变形状
            (batch_size, self.num_heads // group_num * group_num, seq_len, self.head_dim) # 因为，广播后内存就不会再变了，所以可以reshape不影响后续计算, 同一个k-v通过广播共享了 num_heads // group_num 次，其实参数全都共享1个k-v 能够的group_num个q的，复制了 多次

            return x

NameError: name 'torch' is not defined

In [1]:
import torch
import torch.nn as nn

class CausalGroupQueryAttention(torch.nn.Module):
    """
    实现了一个基于组查询的因果注意力机制模块。

    该模块通过将隐藏状态线性变换为查询、键和值，然后在多个注意力头上进行组查询，
    最后通过一个线性变换整合结果，输出最终的注意力结果。特别地，该模块实现了因果注意力，
    确保每个位置只能关注到它之前的位置。

    参数:
    hidden_size (int): 隐藏状态的维度。
    head_num (int): 注意力头的数量。
    group_num (int): 查询组的数量。
    """
    def __init__(self, hidden_size, head_num, group_num):
        super(CausalGroupQueryAttention, self).__init__()
        # 初始化隐藏状态维度、注意力头数量和查询组数量
        self.hidden_size = hidden_size
        self.head_num = head_num
        self.group_num = group_num
        # 计算每个注意力头的维度
        self.head_dim = self.hidden_size // head_num

        # 初始化线性变换层，用于生成查询、键、值和最终的输出整合
        self.q_linear = torch.nn.Linear(hidden_size, hidden_size)
        self.k_linear = torch.nn.Linear(hidden_size, self.group_num * self.head_dim)
        self.v_linear = torch.nn.Linear(hidden_size, self.group_num * self.head_dim)
        self.o_linear = torch.nn.Linear(hidden_size, hidden_size)

    def forward(self, hidden_state, mask=None):
        """
        前向传播函数，实现了组查询因果注意力机制。

        参数:
        hidden_state (Tensor): 输入的隐藏状态，形状为[batch_size, seq_len, hidden_size]。
        mask (Tensor, optional): 注意力掩码，用于指定某些位置的注意力得分为极小值，以避免在计算注意力时考虑这些位置。

        返回:
        Tensor: 注意力机制整合后的输出，形状为[batch_size, seq_len, hidden_size]。
        """
        batch_size, seq_len, _ = hidden_state.size()

        # 对隐藏状态进行线性变换，生成查询、键和值
        query = self.q_linear(hidden_state)
        key = self.k_linear(hidden_state)
        value = self.v_linear(hidden_state)

        # 将查询、键和值分割为多个注意力头
        query = self.split_head(query)
        key = self.split_head(key, self.group_num)
        value = self.split_head(value, self.group_num)

        # 计算注意力分数
        atten_score = torch.matmul(query, key.transpose(-1, -2)) / torch.sqrt(torch.tensor(self.head_dim, dtype=torch.float32))

        # 生成因果掩码，确保每个位置只能看到它之前的位置
        causal_mask = torch.triu(torch.ones((seq_len, seq_len), device=hidden_state.device)).view(1, 1, seq_len, seq_len)
        causal_mask = causal_mask * -1e9

        if mask is not None:
            mask = mask[:, None, None, :] + causal_mask
        else:
            mask = causal_mask

        # 应用掩码atten_score的形状 batch_size, num_heads, seq_len, seq_len
        # 这里mask通过广播机制，加到了atten_score上
        atten_score += mask

        # 计算注意力权重
        atten_weights = torch.softmax(atten_score, dim=-1)

        # 计算加权和
        output = torch.matmul(atten_weights, value)
        output = output.transpose(-1, -2).contiguous().view(batch_size, -1, self.group_num * self.head_dim)

        return self.o_linear(output)

    def split_head(self, x, group_num=None):
        """
        将输入张量分割为多个注意力头。

        参数:
        x (Tensor): 输入张量，形状为[batch_size, seq_len, hidden_size]。
        group_num (int, optional): 查询组的数量。

        返回:
        Tensor: 分割后的张量，形状为[batch_size, head_num, seq_len, head_dim]。
        """
        batch_size, seq_len = x.size()[:2]

        if group_num is not None:
            x = x.view(batch_size, -1, group_num, self.head_dim).transpose(1, 2)
            x = x[:, :, None, :, :].expand(batch_size, group_num, self.head_num // group_num, seq_len, self.head_dim).reshape(batch_size, self.head_num, seq_len, self.head_dim)
        else:
            x = x.view(batch_size, -1, self.head_num, self.head_dim).transpose(1, 2)

        return x

In [2]:
import torch.nn as nn

class GQA(nn.modules):
    def __init__(self, groupnum, headnum, hiddensize):
        super(GQA, self).__init__()
        self.headnum = headnum
        self.groupnum = groupnum
        self.hiddensize = hiddensize
        self.headdim = self.hiddensize // self.headnum

        self.qlinear = nn.Linear(hiddensize, hiddensize)
        self.klinear = nn.Linear(hiddensize, groupnum*self.headdim)
        self.vlinear = nn.Linear(hiddensize, groupnum*self.headdim)
        self.olinear = nn.Linear(hiddensize,hiddensize)

    def forward(self,x,mask=None):
        batchsize = x.size()[0]

        q = self.qlinear(x)
        k = self.klinear(x)
        v = self.vlinear(x)

        q = self.split_head(q)
        k = self.split_head(k,self.groupnum)
        v = self.split_head(v,self.groupnum)

        att_score = torch.matul(q,k.transpose(-1,-2)) // torch.sqrt(torch.tensor(self.headdim))
        if mask:
            att_score += mask * -1e-9

        att_prob = torch.softmax(att_score, dim = -1)
        output = torch.matmul(att_prob,v).transpose(1,2).contiguous().view(batchsize, -1, self.numhead*self.headdim)
        output = self.olinear(output)

        return output

    def split_head(self,x, group_num=None):
        batchsize,seqlen = x.size()[:2]

        if not group_num:
            x = x.view(batchsize,-1,self.headnum,self.headdim).transpose(1,2)
        else:
            x = x.view(batchsize,-1,self.groupnum,self.headdim)
            x = x[:,:,None:,:].extend(batchsize,self.groupnum,self.numhead//self.groupnum,seqlen,self.headdim).reshape(batchsize,self.headnum,seqlen,self.headdim)
        return x

TypeError: module() takes at most 2 arguments (3 given)